In [1]:
import vectorbt as vbt
import numpy as np
import pandas as pd
from numba import njit
import talib
import datetime as dt
import time
from collections import namedtuple
import itertools
import math
from vectorbt.records.nb import col_map_nb
from vectorbt.portfolio import nb as pf_nb, enums as pf_enums
import plotly.io as pio
from numba import njit

In [2]:
@njit(nogil = True)
def enter_position(execution_state_,
                    price_area_,
                    percent_risk_ = 1, 
                    group_ = 0 , column = 0, bar_ = None,
                    direction = "Buy",
                    update_value_ = False, order_records_ = None, 
                    order_counts_ = None, log_records_ = None,
                    log_counts_ = None):
    
    
    if direction == "Buy":
        direction_to_take = pf_enums.Direction.LongOnly
    else:
        direction_to_take = pf_enums.Direction.ShortOnly
        
    Order = pf_nb.order_nb(size = percent_risk_, 
                           size_type = pf_enums.SizeType.ValuePercent100,
                           direction = direction_to_take,
                           price = execution_state_.val_price
                           )

    
    order_result , new_execution_state = vbt.pf_nb.process_order_nb(
                                                        group = group_ ,col = column , i = bar_,
                                                        exec_state = execution_state_,
                                                        order = Order,
                                                        price_area = price_area_,
                                                        update_value = update_value_,
                                                        order_records = order_records_,
                                                        order_counts = order_counts_,
                                                        log_records = log_records_,
                                                        log_counts = log_counts_)
    
    

    
                                                                
    return order_result, new_execution_state
    


In [3]:
@njit(nogil = True)
def close_partial_pos(execution_state_,
                      price_area_,
                      closing_percent = 50, 
                      group_ = 0 , column = 0, bar_ = None,
                      update_value_ = False, order_records_ = None, 
                      order_counts_ = None, log_records_ = None,
                      log_counts_ = None):
    
    
    if execution_state.position > 0 :
        direction_to_take = pf_enums.Direction.LongOnly
    else:
        direction_to_take = pf_enums.Direction.ShortOnly
        
    Order = pf_nb.order_nb(size = -closing_percent, 
                           size_type = pf_enums.SizeType.ValuePercent100,
                           direction = direction_to_take,
                           price = execution_state_.val_price
                           )

    
    order_result , new_execution_state = vbt.pf_nb.process_order_nb(
                                                group = group_ ,col = column , i = bar_,
                                                exec_state = execution_state_,
                                                order = Order,
                                                price_area = price_area_,
                                                update_value = update_value_,
                                                order_records = order_records_,
                                                order_counts = order_counts_,
                                                log_records = log_records_,
                                                log_counts = log_counts_)
    
                                                         
    return order_result, new_execution_state


In [4]:
data = vbt.YFData.download("BTC-USD", end="2022-01-01")
Open = data.get("Open").to_numpy()
High = data.get("High").to_numpy()
Low = data.get("Low").to_numpy()
Close = data.get("Close").to_numpy()

In [5]:
def custom_simulator(open_, high_ , low_ , close_ , IsBullArray,init_cash = 10000):
     
    order_records = np.empty((2663,1), dtype = pf_enums.order_dt)
    order_counts = np.full(1, 0, dtype=np.int_)

    exec_state = pf_enums.ExecState(
                        cash = float(init_cash),
                        position = 0.0,
                        debt = 0.0,
                        locked_cash = 0.0,
                        free_cash = float(init_cash),
                        val_price = np.nan,
                        value = np.nan)
    
    
    for i in range(len(close_)):
        
        price_area = pf_enums.PriceArea(open  = open_[i],
                                            high = high_[i], 
                                            low = low_[i], 
                                            close = close_[i])
        
        value_price = price_area.close
        value = exec_state.cash + (exec_state.position * value_price)
        
        exec_state = pf_enums.ExecState(
                                cash = exec_state.cash,
                                position = exec_state.position,
                                debt = exec_state.debt,
                                locked_cash = exec_state.locked_cash,
                                free_cash = exec_state.free_cash,
                                val_price = value_price,
                                value = value)
        
        if IsBullArray[i] and exec_state.position == 0:
            #CODE THAT WILL ENTER LONG POSITION

            order_result , exec_state = enter_position(
                                                                execution_state_ = exec_state,
                                                                price_area_ = price_area,
                                                                percent_risk_ = 1,
                                                                group_ = 0, column_= 0, bar_ = i,
                                                                direction = 'Buy',
                                                                order_records_= order_records,
                                                                order_counts_ = order_counts)

        elif not IsBullArray[i] and exec_state.position == 0:
            #CODE THAT WILL ENTER SHORT POSITION
            order_result , exec_state = enter_position(
                                                                execution_state_ = exec_state,
                                                                price_area_ = price_area,
                                                                percent_risk_ = 1,
                                                                group_ = 0, column_= 0, bar_ = i,
                                                                direction = 'Sell',
                                                                order_records_= order_records,
                                                                order_counts_ = order_counts)
               

        elif IsBullArray[i] and exec_state.position < 0:
            #CODE THAT WILL CLOSE THE SHORT POSITION HERE
            order_result , exec_state = close_full_position(
                                                                execution_state_ = exec_state,
                                                                price_area_ = price_area,
                                                                group_ = 0, column_= 0, bar_ = i,
                                                                order_records_ = order_records,
                                                                order_counts_  = order_counts)    
            

        elif not IsBullArray[i] and exec_state.position >0:
            order_result , exec_state = close_full_position(
                                                                execution_state_ = exec_state,
                                                                price_area_ = price_area,
                                                                group_ = 0, column_= 0, bar_ = i,
                                                                order_records_= order_records,
                                                                order_counts_ = order_counts)    
        else:
            continue
        
                
    return vbt.nb.repartition_nb(order_records, order_counts)


In [6]:
IsBull = Close > Open
print(IsBull)

[False False False ... False  True False]


In [7]:
custom_simulator(open_ = Open, 
                 high_ = High, 
                 low_  = Low, 
                 close_  = Close, 
                 IsBullArray = IsBull,
                 init_cash = 100000)


AttributeError: module 'vectorbt.portfolio.enums' has no attribute 'ExecState'